## Configuration

In [2]:
from src.widgets import get_config_widgets, select_features
from src.dataset import get_merged_dataset, get_e_price_dataset, standardize_data, adjust_set_sizes

In [3]:
import numpy as np

In [4]:
price_interval =  get_config_widgets()

interactive(children=(FloatSlider(value=0.8, description='Train Set Size', max=0.9, step=0.01), FloatSlider(va…

IntRangeSlider(value=(-74, 222), continuous_update=False, description='Energy Price Interval', max=900, min=-4…

## Data loading

In [5]:
merged_df = get_merged_dataset()
e_price_df = get_e_price_dataset()

min_price, max_price = price_interval.value

merged_ex_out_df = merged_df.copy()[
    (merged_df['Price'] >= min_price) &
    (merged_df['Price'] <= max_price)
]

e_price_ex_out_df = merged_df.copy()[
    (merged_df['Price'] >= min_price) &
    (merged_df['Price'] <= max_price)
]

z_merged_df, z_actual_price_df, actual_price_scaler = standardize_data(merged_df.copy(), e_price_df.copy())
z_merged_ex_out_df, z_actual_ex_out_price_df, actual_price_ex_out_scaler = standardize_data(merged_ex_out_df.copy(), e_price_ex_out_df.copy())



## Feature Selection

In [11]:
features = select_features()
print(features)

Button(description='Confirm Selection', style=ButtonStyle())

['hour', 'dayofyear', 'dayofweek', 'is_public_holiday', 'ma_3_hours', 'ma_6_hours', 'ma_1_days', 'ma_3_days', 'ma_7_days', 'Biomass', 'Hard Coal', 'Hydro', 'Lignite', 'Natural Gas', 'Nuclear', 'Other', 'Pumped storage generation', 'Solar', 'Wind offshore', 'Wind onshore', 'temperature_2m', 'precipitation', 'wind_speed_100m', 'irradiance']


In [7]:
features_df = create_features(merged_df)[FEATURES]

TARGET = merged_df["Price"]
combined_df=pd.concat([features_df, TARGET], axis=1)

train, eval, test, SPLIT_DATE_EVAL, SPLIT_DATE_TEST = split(combined_df, predict_last_day=LAST_DAY_TEST)

NameError: name 'create_features' is not defined

In [ ]:
TARGET = "Price"

X_train = train[FEATURES]
y_train = train[TARGET]

X_test = test[FEATURES]
y_test = test[TARGET]

X_val = eval[FEATURES]
y_val = eval[TARGET]

## Prediction

In [ ]:
linear_regression(X_train, y_train, X_test, y_test)

### Prophet

### XGBoost

In [ ]:
best_params = {
    "base_score": 0.380924762889507,
    "early_stopping_rounds": 215,
    "max_depth": 80,
    "learning_rate": 0.06,
    "n_estimators": 150,
    "gamma": 0.9,
    "min_child_weight": 6,
    "subsample": 0.5719023844219375,
    "colsample_bytree": 0.8815691043681952,
}
y_predicted_mse, model_mse, fi_img_mse = predict_return_mse(best_params, X_train, y_train, X_val, y_val, X_test)
y_predicted_mae, model_mae, fi_img_mae = predict_return_mae(best_params, X_train, y_train, X_val, y_val, X_test)

In [ ]:
for objective in [y_predicted_mse,y_predicted_mae]:
    plot_predicted(test, objective)

buf = io.BytesIO()
plt.savefig(buf, format='png')
plt.show()
plt.close()
buf.seek(0)

ts_image = Image.open(buf)

## Results